## Homework Week 1

In [64]:
import pandas as pd
import numpy as numpy
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [65]:
data_jan = pd.read_parquet("/Users/mdurango/Documents/repositorios/mlops2023/homeworks/data/yellow_tripdata_2022-01.parquet")

In [66]:
data_jan.shape

In [67]:
data_jan.head()

In [68]:
data_jan["duration"] = data_jan["tpep_dropoff_datetime"] - data_jan["tpep_pickup_datetime"]
data_jan.duration = data_jan.duration.apply(lambda td: td.total_seconds() / 60) #take time in minuts

In [69]:
data_jan["duration"].std()

In [70]:
sns.distplot(data_jan["duration"])

In [71]:
filtered_df = data_jan[(data_jan['duration'] >= 1) & (data_jan['duration'] <= 60)]
remaining_records = len(filtered_df)

In [72]:
remaining_records

In [73]:
(remaining_records / data_jan.shape[0])*100

In [74]:
filtered_df.head()

In [78]:
categorical = ["PULocationID", "DOLocationID"]
filtered_df[categorical] = filtered_df[categorical].astype(str)
dv = DictVectorizer()
train_dicts = filtered_df[categorical].to_dict(orient= "records")



In [79]:
train_dicts[:10]

In [80]:
X_train_dv = dv.fit_transform(train_dicts)

In [81]:
X_train_dv.shape

In [82]:
# training model 
lr = LinearRegression()
y_train = filtered_df["duration"].values
lr.fit(X_train_dv, y_train)

In [83]:
y_pred = lr.predict(X_train_dv)
mean_squared_error(y_train, y_pred, squared=False)

In [84]:
def read_process_data(filename):

    if filename.endswith(".parquet"):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df_dicts = df[categorical].to_dict(orient="records")
    X_dv = dv.transform(df_dicts)

    return df, X_dv



In [85]:
data_feb, X_val_dv = read_process_data("/Users/mdurango/Documents/repositorios/mlops2023/homeworks/data/yellow_tripdata_2022-02.parquet")

In [86]:
data_feb

In [90]:
y_val = data_feb["duration"].values


In [89]:
y_val_preds = lr.predict(X_val_dv)

In [91]:
mean_squared_error(y_val, y_val_preds, squared=False)